# Part I. ETL Pipeline for Pre-Processing the Files
Objective: 
Support the company to setup an ETL pipeline to query their dataset using Apache Cassandra 

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages required:

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Here we are creating a list of filepaths to process original event csv data files, the files are located in the event_data folder

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Scripts of Apache Cassandra for data modelling. 

## The data can be accessed in the CSV file titled <font color=red>event_datafile_new.csv</font>.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating a Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace to udacity

In [7]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### In the following section I will create database tables that serve very specific queries. Apache Cassandra queries must model the query, instead of following a relational database schema.


## Below are the 3 business requirements from the dataset: 

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


## Below I create a table following a naming convention that will partially reflect the required elements to be queried for business request No.1
## The primary key is a composite key of the partition key (sessionId) and the clustering column (itemInSession)

In [2]:
## For the first business requirement where the request is:
## Requirement No. 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
### I'm creating a table using as partion key "sessionId" and clustering column "itemInSession"
### Sequence of the columns when running CREATE and INSERT statements will follow the order of the Partition Key \
### and Clustering columns

query = "CREATE TABLE IF NOT EXISTS artist_song_length"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

name 'session' is not defined


## Below I insert the values into the table "artist_song_length"

In [9]:
# Below is the code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## The INSERT statement follows the sequence of CREATE statement where both respect the order of the 
## Primary Key and Clustering columns: 
        query = "INSERT INTO artist_song_length (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## When executing the script we explicitly assign each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

## Below I run a query to retrieve the data of the artist name, song name and lenght of the song, the condition is sessionId = 338 and itemInSession = 4
### The result appears to be:
#### artist name: Faithless 
#### song: Music Matters (Mark Knight Dub)
#### lenght of song: 495.30731201171875

In [5]:
## SELECT statement retrieves the desired data for the specific sessionId and itemInSession:
query = "SELECT artist, song, length FROM artist_song_length WHERE sessionId= 338 and itemInSession= 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## We print the results:
for row in rows:
    print ("artist:", row.artist, "\n" "song title:", row.song, "\n" "lenght:", row.length)

name 'session' is not defined


NameError: name 'rows' is not defined

## Below I create a table (user_play_session) following a naming convention that will partially reflect the required elements to be queried for business request No.2
## The primary key is a composite key of the partition key userId and sessionId and the clustering column (itemInSession)

In [11]:
## For the second business requirement where the request is:
## Requirement No. 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## The same logic of the past query is applied here about the Sequence of the columns when running CREATE and INSERT statements \ 
## will follow the order of the Partition Key dand Clustering columns

query = "CREATE TABLE IF NOT EXISTS user_play_session"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)          

## Below I insert the values into the table "user_play_session"

In [12]:
# Below is the code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## The INSERT statement follows the sequence of CREATE statement where both respect the order of the 
## Primary Key and Clustering columns: 
        query = "INSERT INTO user_play_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## When executing the script we explicitly assign each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

## Below I run a query to retrieve the data of the specific userId = 10 (Sylvie Cruz) where in sessionId 182 she listened to 4 different artists and it's song titles.:
### The result appears to be:
#### Artist: Down To The Bone , Song title: Keep On Keepin' On , First name: Sylvie , Last name: Cruz
#### Artist: Three Drives , Song title: Greece 2000 , First name: Sylvie , Last name: Cruz
#### Artist: Sebastien Tellier , Song title: Kilometer , First name: Sylvie , Last name: Cruz
#### Artist: Lonnie Gordon , Song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , First name: Sylvie , Last name: Cruz

In [13]:
## SELECT statement retrieves the desired data for the specific userId and sessionId:
query = "SELECT artist, song, firstName, lastName FROM user_play_session WHERE userId = 10 and sessionId= 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist:", row.artist, ", Song title:",  row.song, ", First name:", row.firstname,", Last name:", row.lastname)

Down To The Bone Sylvie Cruz Keep On Keepin' On
Three Drives Sylvie Cruz Greece 2000
Sebastien Tellier Sylvie Cruz Kilometer
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


## Below I create a table (users_bysong) following a naming convention that will partially reflect the required elements to be queried for business request No.3
## The primary key is a composite key of the partition key (song) and the clustering column (userId)

In [14]:
## For the third business requirement where the request is:
## Requirement No. 3: Give me every user name (first and last) in my music app history who listened to \ 
## the song 'All Hands Against His Own'
## The same logic of the past query is applied here about the Sequence of the columns when running CREATE and INSERT statements \ 
## will follow the order of the Partition Key dand Clustering columns

query = "CREATE TABLE IF NOT EXISTS users_bysong"
query = query + "(song text, userId int, firstName text, lastName text,  PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

## Below I insert the values into the table "users_bysong"

In [15]:
# Below is the code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## The INSERT statement follows the sequence of CREATE statement where both respect the order of the 
## Primary Key and Clustering columns: 
        query = "INSERT INTO users_bysong (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## When executing the script we explicitly assign each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## Below I run a query to retrieve the data of the specific song = 'All Hands Against His Own':
### The result appears to be:
#### First name:  Jacqueline , Last name:  Lynch
#### First name:  Tegan , Last name:  Levine
#### First name:  Sara , Last name:  Johnson

In [16]:
## SELECT statement retrieves users first and last name where the song was: 'All Hands Against His Own'
query = "SELECT firstName, lastName FROM users_bysong WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("First name: ", row.firstname, ", Last name: ",row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Before shutting down the session and cluster I drop the tables.

In [18]:
query = "drop table artist_song_length"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    query = "drop table user_play_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table users_bysong"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table music_library"


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()